In [1]:

!pip install transformers
!pip install git+https://github.com/openai/whisper.git

from transformers import pipeline
import whisper


ner_pipeline = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")

def predict_entities_pipeline(text):
    """Run the Hugging Face NER pipeline on the input text."""
    return ner_pipeline(text)


whisper_model = whisper.load_model("base")

def transcribe_audio(audio_path):
    """Transcribe audio from a file using Whisper."""
    result = whisper_model.transcribe(audio_path)
    return result["text"]


clinical_terms = {
    "acute myocardial infarction": "DISEASE",
    "aspirin": "MEDICATION",

}

import re

def rule_based_clinical_ner(text):
    """A simple rule-based function to detect predefined clinical terms."""
    entities = []
    lower_text = text.lower()
    for term, label in clinical_terms.items():

        for match in re.finditer(re.escape(term), lower_text):
            start, end = match.span()

            entity_text = text[start:end]
            entities.append({
                "entity_group": label,
                "score": 1.0,
                "word": entity_text,
                "start": start,
                "end": end
            })
    return entities

def predict_entities_clinical(text):
    """
    First, run the Hugging Face NER pipeline.
    If no clinical entities are found, use a rule-based approach.
    """
    pipeline_entities = predict_entities_pipeline(text)

    # Check if any of the pipeline-detected entities match common clinical terms.
    clinical_detected = []
    for ent in pipeline_entities:
        if any(term in ent["word"].lower() for term in clinical_terms):
            clinical_detected.append(ent)

    # If pipeline returns nothing clinical, use rule-based detection.
    if not clinical_detected:
        return rule_based_clinical_ner(text)
    else:
        return clinical_detected

def process_text(text):
    print("Input Text:", text)
    entities = predict_entities_clinical(text)
    print("Extracted Entities:", entities)
    return entities

# Test with a clinical sentence.
clinical_test_text = "Patient diagnosed with acute myocardial infarction and prescribed aspirin."
process_text(clinical_test_text)



  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-b5o1lpfr
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-b5o1lpfr
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu
100%|███████████████████████████████████████| 139M/139M [00:10<00:00, 14.1MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues

Input Text: Patient diagnosed with acute myocardial infarction and prescribed aspirin.
Extracted Entities: [{'entity_group': 'DISEASE', 'score': 1.0, 'word': 'acute myocardial infarction', 'start': 23, 'end': 50}, {'entity_group': 'MEDICATION', 'score': 1.0, 'word': 'aspirin', 'start': 66, 'end': 73}]


[{'entity_group': 'DISEASE',
  'score': 1.0,
  'word': 'acute myocardial infarction',
  'start': 23,
  'end': 50},
 {'entity_group': 'MEDICATION',
  'score': 1.0,
  'word': 'aspirin',
  'start': 66,
  'end': 73}]

In [2]:
clinical_test_text = "Patient diagnosed with acute myocardial infarction and prescribed aspirin."
process_text(clinical_test_text)

Input Text: Patient diagnosed with acute myocardial infarction and prescribed aspirin.
Extracted Entities: [{'entity_group': 'DISEASE', 'score': 1.0, 'word': 'acute myocardial infarction', 'start': 23, 'end': 50}, {'entity_group': 'MEDICATION', 'score': 1.0, 'word': 'aspirin', 'start': 66, 'end': 73}]


[{'entity_group': 'DISEASE',
  'score': 1.0,
  'word': 'acute myocardial infarction',
  'start': 23,
  'end': 50},
 {'entity_group': 'MEDICATION',
  'score': 1.0,
  'word': 'aspirin',
  'start': 66,
  'end': 73}]

In [3]:

!pip install transformers datasets torch


from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

data = {
    "text": [
        "Patient diagnosed with acute myocardial infarction and prescribed aspirin.",
        "Patient presents with symptoms of pneumonia and is given antibiotics.",
        "The patient shows no signs of cardiovascular disease.",
        "Patient diagnosed with diabetes and advised to follow a low-sugar diet.",
        "Patient has a history of asthma and requires inhalers."
    ],
    "label": [0, 1, 2, 3, 1]  # Labels corresponding to the classes defined above.
}


label2name = {0: "Cardiovascular", 1: "Respiratory", 2: "None", 3: "Metabolic"}


dataset = Dataset.from_dict(data)


split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]


model_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label2name))

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Rename the label column to "labels" for Trainer compatibility.
train_dataset = train_dataset.rename_column("label", "labels")
eval_dataset = eval_dataset.rename_column("label", "labels")

# Set the format of the dataset to PyTorch tensors.
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


training_args = TrainingArguments(
    output_dir="./clinical_classifier",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

print("Training the classifier...")
trainer.train()


def predict_disease(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    return label2name[predicted_label]


test_text = "Patient diagnosed with acute myocardial infarction and prescribed aspirin."
print("Predicted Disease:", predict_disease(test_text))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.0 MB/s eta 0:00:00


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: namanchawla819 (namanchawla819-bits-pilani-dubai-campus) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,1.036290
2,No log,1.126096
3,No log,1.116457


Predicted Disease: Respiratory
